In [ ]:
import ee
import geopandas as gpd
from shapely.geometry import shape
import geemap
import folium

def get_basin_and_rivers(hydrobasins_col, rivers_col, coordinates):
    point = ee.Geometry.Point(coordinates)
    basin = hydrobasins_col.filterBounds(point)
    rivers = rivers_col.filterBounds(basin.geometry())
    print('Intersecting Basin:', basin.getInfo())
    return basin, rivers

def get_river_elev_slope(dem_img, basin, rivers):
    def get_min_max(basin, var, label):
        var_min_max = var.reduceRegion(
            reducer=ee.Reducer.minMax(),
            geometry=basin,  
            scale=30,
            maxPixels=1e9
        )
        print(f'{label} range:', var_min_max.getInfo())
        return
    elevation_rivers = dem_img.clip(rivers)
    slopes_rivers = ee.Terrain.slope(elevation_rivers)
    get_min_max(basin, elevation_rivers, 'Elevation')
    get_min_max(basin, slopes_rivers, 'Slopes')
    return elevation_rivers, slopes_rivers

def map_basin_river(Map, basin, rivers):
    visualization_basin = {
        'color': 'yellow', 
        'strokeWidth': 1
    }
    visualization_rivers = {
        'color': 'red', 
        'strokeWidth': 1
    }
    Map.addLayer(basin, visualization_basin, 'Basin')
    Map.addLayer(rivers, visualization_rivers, 'Rivers')
    Map.centerObject(basin, 12)
    return Map

def map_elevation_slope(elevation_rivers, slopes_rivers, Map):
    visualization_elevation = {
        'min': 0,  
        'max': 5000,  
        'palette': ['blue', 'green', 'yellow', 'brown', 'white']
    }
    visualization_slopes = {
    'min': 11,  
    'max': 28,  
    'palette': ['blue', 'green', 'yellow', 'brown', 'white']  
    }
    Map.addLayer(elevation_rivers, visualization_elevation, 'Elevation rivers')
    Map.addLayer(slopes_rivers, visualization_slopes, 'Slope rivers')
    return Map

def feature_collection_to_gdf(feature_collection, crs):
    # Step 1: Get the features from the FeatureCollection as a dictionary
    features = feature_collection.getInfo()['features']
    # Step 2: Convert the dictionary into a list of geometries and properties
    data = []
    for feature in features:
        # Safely retrieve geometry and properties
        geometry = feature.get('geometry', {})
        properties = feature.get('properties', {})  # Handle missing properties
        if geometry:
            # Convert the geometry to a shapely object
            shapely_geom = shape(geometry)
            data.append({'geometry': shapely_geom, **properties})
    # Step 3: Convert the list of dictionaries into a GeoDataFrame
    gdf = gpd.GeoDataFrame(data)
    if crs:
        gdf.set_crs(crs, inplace=True)
    return gdf

def get_length(rivers, gdf):
    # Function to calculate the straight-line distance for each reach
    def calculate_straight_line_distance(rivers, reach_id):
        # Filter the feature collection by REACH_ID
        reach_feature = rivers.filter(ee.Filter.eq('REACH_ID', reach_id)).first()
        
        # Get the coordinates of the start and end points; IS THIS RIGHT?
        coordinates = reach_feature.geometry().coordinates()
        start_point = ee.Geometry.Point(coordinates.get(0))
        end_point = ee.Geometry.Point(coordinates.get(-1))
        
        # Calculate the straight-line distance in kilometers
        straight_line_distance = start_point.distance(end_point).divide(1000)  # Convert meters to kilometers
        
        return straight_line_distance.getInfo()
    
    gdf['STRAIGHT_LINE_DISTANCE_KM'] = gdf.apply(lambda row: calculate_straight_line_distance(rivers, row['REACH_ID']), axis=1)
    return gdf

def get_mean_elevation(elevation, rivers, gdf):
    # Function to calculate the straight-line distance for each reach
    def calculate_mean_elevation(rivers, reach_id):
        reach_feature = ee.Feature(rivers.filter(ee.Filter.eq('REACH_ID', reach_id)).first())
        geometry = reach_feature.geometry()
        clipped_elevation = elevation.clip(geometry)

        mean_elevation = clipped_elevation.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=geometry,
            scale=30
        ).get('elevation')
        return mean_elevation.getInfo()
    
    gdf['ELEVATION'] = gdf.apply(lambda row: calculate_mean_elevation(rivers, row['REACH_ID']), axis=1)
    return gdf

def get_mean_slope(elevation, rivers, gdf):
    # Function to calculate the straight-line distance for each reach
    def calculate_mean_slope(rivers, slope, reach_id):
        reach_feature = ee.Feature(rivers.filter(ee.Filter.eq('REACH_ID', reach_id)).first())
        geometry = reach_feature.geometry()
        clipped_slope = slope.clip(geometry)

        mean_slope = clipped_slope.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=geometry,
            scale=30
        ).get('slope')
        return mean_slope.getInfo()
    slope = ee.Terrain.slope(elevation)
    gdf['SLOPE'] = gdf.apply(lambda row: calculate_mean_slope(rivers, slope, row['REACH_ID']), axis=1)
    return gdf

def get_sinuosity(gdf):
    gdf['SINUOSITY'] = gdf['LENGTH_KM'] / gdf['STRAIGHT_LINE_DISTANCE_KM']
    # gdf['SINUOSITY'] = gdf['SINUOSITY'].apply(lambda x: max(x, 1)) #TODO: Check with Natashs if sinuosity always needs to be over 1.
    return gdf

def plot_sinuosity(gdf, threshold):
    def my_colormap(value):  
        if value >= threshold:
            return "green"
        return "red"
    return gdf.explore(color=gdf['SINUOSITY'].apply(my_colormap))

def plot_csi(gdf, threshold):
    def my_colormap(value):  
        if value >= threshold:
            return "green"
        return "red"
    return gdf.explore(color=gdf['CSI'].apply(my_colormap))

def plot_fpz(gdf):
    def my_colormap(value):  
        if value == 'Lowland Alluvial':
            return "green"
        elif value == 'Open-valley Lowland':
            return "yellow"
        elif value == 'Open-Valley Upland':
            return "orange"
        elif value == 'Upland Constrained':
            return "pink"
        return "red"
    return gdf.explore(color=gdf['FPZs'].apply(my_colormap))

def get_classification(gdf):
    classifications = [
        {
            'name': 'Lowland Alluvial',
            'ELEVATION': lambda ele: ele < 200,
            'SLOPE': lambda slo: slo < 2,
            'SINUOSITY': lambda sin: sin > 1.4
        },
        {
            'name': 'Open-valley Lowland',
            'ELEVATION': lambda ele: ele < 200,
            'SLOPE': lambda slo:  2 <= slo <= 4,
            'SINUOSITY': lambda sin: sin > 1.2
        },
        {
            'name': 'Open-Valley Upland',
            'ELEVATION': lambda ele: 200 <= ele <= 800,
            'SLOPE': lambda slo: 2 <= slo <= 4,
            'SINUOSITY': lambda sin:  1.0 <= sin <= 1.2
        },
        {
            'name': 'Upland Constrained',
            'ELEVATION': lambda ele: ele > 800,
            'SLOPE': lambda slo: slo > 4,
            'SINUOSITY': lambda sin: 1.0 <= sin <= 1.1
        }
    ]

    def classify_row(row):
        for classification in classifications:
            if (classification['ELEVATION'](row['ELEVATION']) and
                classification['SLOPE'](row['SLOPE']) and
                classification['SINUOSITY'](row['SINUOSITY'])):
                return classification['name']
        return 'Unclassified'  

    gdf['FPZs'] = gdf.apply(classify_row, axis=1)
    return gdf

def get_fpz(hydrobasins_col, rivers_col, dem_img, coordinates):
    basin, rivers = get_basin_and_rivers(hydrobasins_col, rivers_col, coordinates)
    gdf_rivers = feature_collection_to_gdf(rivers, crs="EPSG:4326") 
    gdf_rivers = get_length(rivers, gdf_rivers)
    gdf_rivers = get_sinuosity(gdf_rivers)
    gdf_rivers = get_mean_elevation(dem_img, rivers, gdf_rivers)
    gdf_rivers = get_mean_slope(dem_img, rivers, gdf_rivers)
    gdf_rivers = get_classification(gdf_rivers)
    gdf_rivers = gdf_rivers[['REACH_ID', 'LENGTH_KM','STRAIGHT_LINE_DISTANCE_KM','SINUOSITY','ELEVATION','SLOPE', 'FPZs', 'CSI', 'CSI_FF1', 'CSI_FF2', 'CSI_FFID', 'geometry']]
    return basin, rivers, gdf_rivers 

def plot_elevation_slope_river_network(dem_img, basin, rivers):
    elevation_rivers, slopes_rivers = get_river_elev_slope(dem_img, basin, rivers)
    Map = geemap.Map()
    Map = map_basin_river(Map, basin, rivers)
    Map = map_elevation_slope(elevation_rivers, slopes_rivers, Map)
    return Map


In [15]:
# ee.Authenticate()
ee.Initialize()

hydrobasins_col = ee.FeatureCollection('WWF/HydroSHEDS/v1/Basins/hybas_12')
rivers_col = ee.FeatureCollection("WWF/HydroSHEDS/v1/FreeFlowingRivers")
dem_img = ee.Image('USGS/SRTMGL1_003').select('elevation')

coord_Aracataca = [-73.9122858271122, 10.649659492838959]
coord_Fundacion = [-74.1775961943024, 10.507908590834537]

basin, rivers, gdf_rivers = get_fpz(hydrobasins_col, rivers_col, dem_img, coord_Aracataca)
plot_elevation_slope_river_network(dem_img, basin, rivers)
# plot_sinuosity(gdf_rivers,1)
# plot_cci(gdf_rivers, 99)
# plot_fpz(gdf_rivers)

Intersecting Basin: {'type': 'FeatureCollection', 'columns': {'COAST': 'Integer', 'DIST_MAIN': 'Float', 'DIST_SINK': 'Float', 'ENDO': 'Integer', 'HYBAS_ID': 'Long', 'MAIN_BAS': 'Long', 'NEXT_DOWN': 'Long', 'NEXT_SINK': 'Long', 'ORDER': 'Integer', 'PFAF_ID': 'Long', 'SORT': 'Long', 'SUB_AREA': 'Float', 'UP_AREA': 'Float', 'system:index': 'String'}, 'version': 1576573319542367, 'id': 'WWF/HydroSHEDS/v1/Basins/hybas_12', 'properties': {'system:asset_size': 904941518}, 'features': [{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-73.98750044426117, 10.654165844780604], [-73.98714821210419, 10.647575307027687], [-73.97951865842607, 10.63992346047671], [-73.97916641163486, 10.624998824857956], [-73.97881416419833, 10.614243493816522], [-73.96701978013404, 10.602422441335724], [-73.96666751046924, 10.600001135333729], [-73.95923418347893, 10.600661093286906], [-73.95742827333319, 10.607688597476326], [-73.94067993067247, 10.608901461437005], [-73.92708411104086, 10.59628

Map(center=[10.660424610529107, -73.90670328725965], controls=(WidgetControl(options=['position', 'transparent…

In [28]:
import ee
import geemap

# Initialize the Earth Engine module
ee.Initialize()

# Load the MERIT Hydro dataset
dataset = ee.Image('MERIT/Hydro/v1_0_1')
dataset = dataset.clip(basin)

# Define the visualization parameters
visualization = {
    'bands': ['viswth'],
}

# Create a map centered at the specified location and zoom level
Map = geemap.Map()
Map.centerObject(basin, 12)
# Add the dataset layer with visualization parameters to the map
Map.addLayer(dataset, visualization, 'River width')
Map = map_basin_river(Map, basin, rivers)

# Display the map
Map


Map(center=[10.660424610529107, -73.90670328725965], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
# Load the necessary datasets
hydrobasins = ee.FeatureCollection('WWF/HydroSHEDS/v1/Basins/hybas_12')
rivers = ee.FeatureCollection("WWF/HydroSHEDS/v1/FreeFlowingRivers")
srtm = ee.Image('USGS/SRTMGL1_003')

# Define coordinates (Aracataca example)
coord_Aracataca = [-73.9122858271122, 10.649659492838959]
coord_Fundacion = [-74.1775961943024, 10.507908590834537]
coord = coord_Aracataca 

# Create a point geometry from the coordinates
point = ee.Geometry.Point(coord)

# Filter hydrobasins and rivers based on the point
intersecting_basin = hydrobasins.filterBounds(point)
basin_limits = intersecting_basin.geometry()
intersecting_rivers = rivers.filterBounds(basin_limits)

# Clip the elevation image to the basin limits
intersecting_elevation = srtm.select('elevation').clip(intersecting_rivers)

# Calculate slope from the elevation
slopes = ee.Terrain.slope(intersecting_elevation)

# Print the intersecting basin information
print('Intersecting Basin:', intersecting_basin.getInfo())

# Visualization parameters
visualization_basin = {
    'color': 'yellow', 
    'strokeWidth': 1
}
visualization_rivers = {
    'color': 'red', 
    'strokeWidth': 1
}
visualization_elevation = {
    'min': 0,  
    'max': 5000,  
    'palette': ['blue', 'green', 'yellow', 'brown', 'white']
}

visualization_slopes = {
  'min': 11,  
  'max': 28,  
  'palette': ['blue', 'green', 'yellow', 'brown', 'white']  
}

# Get the min and max values for the elevation
elevation_min_max = intersecting_elevation.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=basin_limits,  
    scale=30,
    maxPixels=1e9
)
print('Elevation range:', elevation_min_max.getInfo())

# Get the min and max values for the slope
slope_min_max = slopes.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=basin_limits,  
    scale=30,
    maxPixels=1e9
)
print('Slope range:', slope_min_max.getInfo())

# Create a map
Map = geemap.Map()

# Add the layers to the map
Map.addLayer(intersecting_basin, visualization_basin, 'Intersecting Basin')
Map.addLayer(intersecting_elevation, visualization_elevation, 'Intersecting Elevation')
Map.addLayer(slopes, visualization_slopes, 'Slope')
Map.addLayer(intersecting_rivers, visualization_rivers, 'Intersecting Rivers')

# Center the map on the basin
Map.centerObject(basin_limits, 12)

# Display the map
Map



Intersecting Basin: {'type': 'FeatureCollection', 'columns': {'COAST': 'Integer', 'DIST_MAIN': 'Float', 'DIST_SINK': 'Float', 'ENDO': 'Integer', 'HYBAS_ID': 'Long', 'MAIN_BAS': 'Long', 'NEXT_DOWN': 'Long', 'NEXT_SINK': 'Long', 'ORDER': 'Integer', 'PFAF_ID': 'Long', 'SORT': 'Long', 'SUB_AREA': 'Float', 'UP_AREA': 'Float', 'system:index': 'String'}, 'version': 1576573319542367, 'id': 'WWF/HydroSHEDS/v1/Basins/hybas_12', 'properties': {'system:asset_size': 904941518}, 'features': [{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-73.98750044426117, 10.654165844780604], [-73.98714821210419, 10.647575307027687], [-73.97951865842607, 10.63992346047671], [-73.97916641163486, 10.624998824857956], [-73.97881416419833, 10.614243493816522], [-73.96701978013404, 10.602422441335724], [-73.96666751046924, 10.600001135333729], [-73.95923418347893, 10.600661093286906], [-73.95742827333319, 10.607688597476326], [-73.94067993067247, 10.608901461437005], [-73.92708411104086, 10.59628

Map(center=[10.660424610529107, -73.90670328725965], controls=(WidgetControl(options=['position', 'transparent…

In [147]:
# Goal is to classify the river into Functional Process Zones by using the parameters: 
# elevation, slope, sinuosity, average valley width to valley flood width ratio
#Author: Natasha Y. Flores

import ee
import geemap
import folium

# Initialize the Earth Engine module.
# ee.Authenticate()
ee.Initialize()

# Load HydroBASINS data
hydrobasins = ee.FeatureCollection("WWF/HydroSHEDS/v1/Basins/hybas_12")

# Filter for the specified HYBAS_ID and name it aracataca_basin
aracataca_basin = hydrobasins.filter(ee.Filter.eq('HYBAS_ID', 6120977000))

# Print the details of the aracataca_basin
aracataca_basin_info = aracataca_basin.getInfo()
print(aracataca_basin_info)

# Load HydroSHEDS HydroRIVERS dataset
rivers = ee.FeatureCollection("WWF/HydroSHEDS/v1/FreeFlowingRivers")

#Clip the river network to Aracataca
aracataca_rivers = rivers.filterBounds(aracataca_basin.geometry())

# Load SRTM elevation data and clip to the basin
srtm = ee.Image('USGS/SRTMGL1_003').clip(aracataca_rivers.geometry())

# Select the elevation band
elevation = srtm.select('elevation')

# Show min and max of elevation
elevation_min_max = elevation.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aracataca_basin.geometry(),
    scale=30,
    maxPixels=1e9
)
print('elevation range:', elevation_min_max.getInfo())

# Calculate slope from SRTM elevation data
slope = ee.Terrain.slope(elevation)

# Show min and max of slope
slope_min_max = slope.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aracataca_basin.geometry(),
    scale=30,
    maxPixels=1e9
)
print('Slope range:', slope_min_max.getInfo())

# Define a function to add Earth Engine layers to the map
def add_ee_layer(self, ee_object, vis_params, name):
    map_id_dict = ee_object.getMapId(vis_params)
    folium.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add EE drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object
Map = folium.Map(location=[10.643973, -73.886836], zoom_start=10)

# Add the layers to the map with distinctive visualization parameters
Map.add_ee_layer(elevation, {'min': 0, 'max': 3000, 'palette': ['blue', 'green', 'red']}, 'SRTM Elevation')
Map.add_ee_layer(slope, {'min': 0, 'max': 50, 'palette': ['yellow', 'orange', 'red']}, 'Slope')
Map.add_ee_layer(aracataca_basin, {'color': 'blue'}, 'aracataca Basin')
Map.add_ee_layer(aracataca_rivers, {'color': 'cyan'}, 'aracataca Rivers')

# Add a layer control panel to the map
# Map.add_child(folium.LayerControl())

# Display the map
# Map

{'type': 'FeatureCollection', 'columns': {'COAST': 'Integer', 'DIST_MAIN': 'Float', 'DIST_SINK': 'Float', 'ENDO': 'Integer', 'HYBAS_ID': 'Long', 'MAIN_BAS': 'Long', 'NEXT_DOWN': 'Long', 'NEXT_SINK': 'Long', 'ORDER': 'Integer', 'PFAF_ID': 'Long', 'SORT': 'Long', 'SUB_AREA': 'Float', 'UP_AREA': 'Float', 'system:index': 'String'}, 'version': 1576573319542367, 'id': 'WWF/HydroSHEDS/v1/Basins/hybas_12', 'properties': {'system:asset_size': 904941518}, 'features': [{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-73.98750044426117, 10.654165844780604], [-73.98714821210419, 10.647575307027687], [-73.97951865842607, 10.63992346047671], [-73.97916641163486, 10.624998824857956], [-73.97881416419833, 10.614243493816522], [-73.96701978013404, 10.602422441335724], [-73.96666751046924, 10.600001135333729], [-73.95923418347893, 10.600661093286906], [-73.95742827333319, 10.607688597476326], [-73.94067993067247, 10.608901461437005], [-73.92708411104086, 10.596286680489118], [-73.91

In [148]:
#Start of sinuosity calculation (sinuosity = Length of river channel/Length of straight line distance)
# Reduce the collection to get unique REACH_ID and their LENGTH_KM
unique_reach_ids = aracataca_rivers.reduceColumns(
    reducer=ee.Reducer.toList().repeat(2),
    selectors=['REACH_ID', 'LENGTH_KM' ]
).getInfo()
print(unique_reach_ids)

# Extract the results
reach_ids  = unique_reach_ids['list'][0]
lengths_km = unique_reach_ids['list'][1]

# Combine REACH_ID and LENGTH_KM into a list of dictionaries
reach_length_list = [{'REACH_ID': reach_id, 'LENGTH_KM': length_km} for reach_id, length_km in zip(reach_ids, lengths_km)]

# Print the results
for reach in reach_length_list:
    print(f"REACH_ID: {reach['REACH_ID']}, LENGTH_KM: {reach['LENGTH_KM']}")

{'list': [[60003284, 60003308, 60003348, 60003428, 60003038, 60003180, 60003037, 60003307, 60003532, 60003283, 60003455, 60003265, 60003363, 60003452, 60003266, 60003306, 60003347, 60003501, 60003207, 60003364, 60003407, 60003429, 60003453, 60003454, 60003500, 60003531, 60003208, 60003499, 60003427], [3.509, 0.65, 2.665, 1.366, 1.158, 3.152, 0.683, 0.974, 1.994, 1.345, 3.998, 1.624, 2.263, 1.622, 2.548, 1.299, 1.113, 2.543, 1.105, 3.802, 2.673, 1.105, 1.366, 1.105, 2.665, 1.105, 1.762, 1.158, 1.622]]}
REACH_ID: 60003284, LENGTH_KM: 3.509
REACH_ID: 60003308, LENGTH_KM: 0.65
REACH_ID: 60003348, LENGTH_KM: 2.665
REACH_ID: 60003428, LENGTH_KM: 1.366
REACH_ID: 60003038, LENGTH_KM: 1.158
REACH_ID: 60003180, LENGTH_KM: 3.152
REACH_ID: 60003037, LENGTH_KM: 0.683
REACH_ID: 60003307, LENGTH_KM: 0.974
REACH_ID: 60003532, LENGTH_KM: 1.994
REACH_ID: 60003283, LENGTH_KM: 1.345
REACH_ID: 60003455, LENGTH_KM: 3.998
REACH_ID: 60003265, LENGTH_KM: 1.624
REACH_ID: 60003363, LENGTH_KM: 2.263
REACH_ID: 600

In [149]:
# Function to calculate the straight-line distance for each reach
def calculate_straight_line_distance(reach_id):
    # Filter the feature collection by REACH_ID
    reach_feature = aracataca_rivers.filter(ee.Filter.eq('REACH_ID', reach_id)).first()
    
    # Get the coordinates of the start and end points; IS THIS RIGHT?
    coordinates = reach_feature.geometry().coordinates()
    start_point = ee.Geometry.Point(coordinates.get(0))
    end_point = ee.Geometry.Point(coordinates.get(-1))
    
    # Calculate the straight-line distance in kilometers
    straight_line_distance = start_point.distance(end_point).divide(1000)  # Convert meters to kilometers
    
    return straight_line_distance.getInfo()

# Append straight-line distances to the reach_length_list
for reach in reach_length_list:
    reach['STRAIGHT_LINE_DISTANCE_KM'] = calculate_straight_line_distance(reach['REACH_ID'])

# Print the results
for reach in reach_length_list:
    print(f"REACH_ID: {reach['REACH_ID']}, LENGTH_KM: {reach['LENGTH_KM']}, STRAIGHT_LINE_DISTANCE_KM: {reach['STRAIGHT_LINE_DISTANCE_KM']}")


REACH_ID: 60003284, LENGTH_KM: 3.509, STRAIGHT_LINE_DISTANCE_KM: 2.592474182207558
REACH_ID: 60003308, LENGTH_KM: 0.65, STRAIGHT_LINE_DISTANCE_KM: 0.6486495434076742
REACH_ID: 60003348, LENGTH_KM: 2.665, STRAIGHT_LINE_DISTANCE_KM: 2.4586942533891714
REACH_ID: 60003428, LENGTH_KM: 1.366, STRAIGHT_LINE_DISTANCE_KM: 1.3680906125703352
REACH_ID: 60003038, LENGTH_KM: 1.158, STRAIGHT_LINE_DISTANCE_KM: 1.152182712876476
REACH_ID: 60003180, LENGTH_KM: 3.152, STRAIGHT_LINE_DISTANCE_KM: 2.911936217672379
REACH_ID: 60003037, LENGTH_KM: 0.683, STRAIGHT_LINE_DISTANCE_KM: 0.683399155571331
REACH_ID: 60003307, LENGTH_KM: 0.974, STRAIGHT_LINE_DISTANCE_KM: 0.9722828257091025
REACH_ID: 60003532, LENGTH_KM: 1.994, STRAIGHT_LINE_DISTANCE_KM: 1.8528023015618071
REACH_ID: 60003283, LENGTH_KM: 1.345, STRAIGHT_LINE_DISTANCE_KM: 1.239183419846355
REACH_ID: 60003455, LENGTH_KM: 3.998, STRAIGHT_LINE_DISTANCE_KM: 3.4506035535381883
REACH_ID: 60003265, LENGTH_KM: 1.624, STRAIGHT_LINE_DISTANCE_KM: 1.62055384200297


In [150]:
# Calculate sinuosity for each reach
for reach in reach_length_list:
    reach['SINUOSITY'] = reach['LENGTH_KM'] / reach['STRAIGHT_LINE_DISTANCE_KM']

# Print the results
for reach in reach_length_list:
    print(f"REACH_ID: {reach['REACH_ID']}, LENGTH_KM: {reach['LENGTH_KM']}, STRAIGHT_LINE_DISTANCE_KM: {reach['STRAIGHT_LINE_DISTANCE_KM']}, SINUOSITY: {reach['SINUOSITY']}")

REACH_ID: 60003284, LENGTH_KM: 3.509, STRAIGHT_LINE_DISTANCE_KM: 2.592474182207558, SINUOSITY: 1.353533247923031
REACH_ID: 60003308, LENGTH_KM: 0.65, STRAIGHT_LINE_DISTANCE_KM: 0.6486495434076742, SINUOSITY: 1.0020819510412837
REACH_ID: 60003348, LENGTH_KM: 2.665, STRAIGHT_LINE_DISTANCE_KM: 2.4586942533891714, SINUOSITY: 1.0839086626271028
REACH_ID: 60003428, LENGTH_KM: 1.366, STRAIGHT_LINE_DISTANCE_KM: 1.3680906125703352, SINUOSITY: 0.9984718756556575
REACH_ID: 60003038, LENGTH_KM: 1.158, STRAIGHT_LINE_DISTANCE_KM: 1.152182712876476, SINUOSITY: 1.0050489276210375
REACH_ID: 60003180, LENGTH_KM: 3.152, STRAIGHT_LINE_DISTANCE_KM: 2.911936217672379, SINUOSITY: 1.0824412914234478
REACH_ID: 60003037, LENGTH_KM: 0.683, STRAIGHT_LINE_DISTANCE_KM: 0.683399155571331, SINUOSITY: 0.9994159261566584
REACH_ID: 60003307, LENGTH_KM: 0.974, STRAIGHT_LINE_DISTANCE_KM: 0.9722828257091025, SINUOSITY: 1.0017661263219837
REACH_ID: 60003532, LENGTH_KM: 1.994, STRAIGHT_LINE_DISTANCE_KM: 1.8528023015618071, S

In [151]:
# Update the aracataca_rivers layer with sinuosity values
def add_sinuosity(feature):
    reach_id = feature.get('REACH_ID')
    sinuosity = next((reach['SINUOSITY'] for reach in reach_length_list if reach['REACH_ID'] == reach_id), None)
    return feature.set({'SINUOSITY': sinuosity})

aracataca_rivers_with_sinuosity = aracataca_rivers.map(add_sinuosity)

# Define a function to add Earth Engine layers to the map
def add_ee_layer(self, ee_object, vis_params, name):
    map_id_dict = ee_object.getMapId(vis_params)
    folium.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add EE drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object
Map = folium.Map(location=[10.643973, -73.886836], zoom_start=10)

# Function to style features based on sinuosity values
def style_function(feature):
    sinuosity = feature['properties'].get('SINUOSITY', 1)  # Default to 1 if SINUOSITY is not found
    if sinuosity >= 1:
        return {'color': 'cyan'}
    else:
        return {'color': 'red'}

# Function to add labels to features
def add_labels(feature):
    reach_id = feature['properties'].get('REACH_ID', 'Unknown')
    return folium.Popup(f"REACH_ID: {reach_id}")

# Add the Aracataca rivers layer with sinuosity color coding and labels
aracataca_rivers_geojson = aracataca_rivers_with_sinuosity.getInfo()
for feature in aracataca_rivers_geojson['features']:
    styled_feature = style_function(feature)
    label = add_labels(feature)
    folium.GeoJson(
        feature,
        style_function=lambda x: styled_feature,
        tooltip=label
    ).add_to(Map)

# Add a layer control panel to the map
Map.add_child(folium.LayerControl())

# Display the map
Map

In [152]:
# Extracting connectivity attributes
# Reduce the collection to get unique REACH_ID and their CSI, CSI_FF1, CSI_FF2, and CSI_FFID
csi_data = aracataca_rivers_with_sinuosity.reduceColumns(
    reducer=ee.Reducer.toList().repeat(5),
    selectors=['REACH_ID', 'CSI', 'CSI_FF1', 'CSI_FF2', 'CSI_FFID']
).getInfo()

# Extract the results
reach_ids = csi_data['list'][0]
csi_values = csi_data['list'][1]
csi_ff1_values = csi_data['list'][2]
csi_ff2_values = csi_data['list'][3]
csi_ffid_values = csi_data['list'][4]

# Combine the extracted data into a list of dictionaries
csi_list = [{'REACH_ID': reach_id, 'CSI': csi, 'CSI_FF1': csi_ff1, 'CSI_FF2': csi_ff2, 'CSI_FFID': csi_ffid} 
            for reach_id, csi, csi_ff1, csi_ff2, csi_ffid in zip(reach_ids, csi_values, csi_ff1_values, csi_ff2_values, csi_ffid_values)]

# Print the results
for reach in csi_list:
    print(f"REACH_ID: {reach['REACH_ID']}, CSI: {reach['CSI']}, CSI_FF1: {reach['CSI_FF1']}, CSI_FF2: {reach['CSI_FF2']}, CSI_FFID: {reach['CSI_FFID']}")

REACH_ID: 60003284, CSI: 100, CSI_FF1: 1, CSI_FF2: 1, CSI_FFID: 2803895
REACH_ID: 60003308, CSI: 100, CSI_FF1: 1, CSI_FF2: 1, CSI_FFID: 2803895
REACH_ID: 60003348, CSI: 100, CSI_FF1: 1, CSI_FF2: 1, CSI_FFID: 2803895
REACH_ID: 60003428, CSI: 100, CSI_FF1: 1, CSI_FF2: 1, CSI_FFID: 2803895
REACH_ID: 60003038, CSI: 100, CSI_FF1: 1, CSI_FF2: 1, CSI_FFID: 2803977
REACH_ID: 60003180, CSI: 100, CSI_FF1: 1, CSI_FF2: 1, CSI_FFID: 2803977
REACH_ID: 60003037, CSI: 100, CSI_FF1: 1, CSI_FF2: 1, CSI_FFID: 2803981
REACH_ID: 60003307, CSI: 100, CSI_FF1: 1, CSI_FF2: 1, CSI_FFID: 2803966
REACH_ID: 60003532, CSI: 100, CSI_FF1: 1, CSI_FF2: 1, CSI_FFID: 2803937
REACH_ID: 60003283, CSI: 100, CSI_FF1: 1, CSI_FF2: 1, CSI_FFID: 2803972
REACH_ID: 60003455, CSI: 100, CSI_FF1: 1, CSI_FF2: 1, CSI_FFID: 2803955
REACH_ID: 60003265, CSI: 100, CSI_FF1: 1, CSI_FF2: 1, CSI_FFID: 2803971
REACH_ID: 60003363, CSI: 100, CSI_FF1: 1, CSI_FF2: 1, CSI_FFID: 2803967
REACH_ID: 60003452, CSI: 100, CSI_FF1: 1, CSI_FF2: 1, CSI_FFID: 

In [153]:
# Define a function to add layers to the map
def add_ee_layer(self, ee_object, vis_params, name):
    map_id_dict = ee_object.getMapId(vis_params)
    folium.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add EE drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object
Map = folium.Map(location=[10.643973, -73.886836], zoom_start=10)

# Function to style features based on CSI values
def style_function(feature):
    csi = feature['properties']['CSI']
    if csi >= 99:
        return {'color': 'green'}
    else:
        return {'color': 'red'}

# Function to add labels to features
def add_labels(feature):
    reach_id = feature['properties']['REACH_ID']
    return folium.Popup(f"REACH_ID: {reach_id}")

# Add the Aracataca rivers layer with CSI color coding and labels
aracataca_rivers_geojson = aracataca_rivers_with_sinuosity.getInfo()
for feature in aracataca_rivers_geojson['features']:
    styled_feature = style_function(feature)
    label = add_labels(feature)
    folium.GeoJson(
        feature,
        style_function=lambda x: styled_feature,
        tooltip=label
    ).add_to(Map)

# Add a layer control panel to the map
Map.add_child(folium.LayerControl())

# Display the map
Map

In [154]:
# Check the list in the memory
## Print the contents of reach_length_list to debug
#for reach in reach_length_list:
#    print(reach)

# Load the elevation and slope datasets
elevation = ee.Image("USGS/SRTMGL1_003")
slope = ee.Terrain.slope(elevation)

# Function to calculate mean elevation and slope for each reach
def calculate_mean_elevation_slope(reach_feature):
    geometry = reach_feature.geometry()
    
    # Clip the elevation and slope layers to the reach geometry
    clipped_elevation = elevation.clip(geometry)
    clipped_slope = slope.clip(geometry)
    
    # Calculate mean elevation for the reach
    mean_elevation = clipped_elevation.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=geometry,
        scale=30
    ).get('elevation')
    
    # Calculate mean slope for the reach
    mean_slope = clipped_slope.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=geometry,
        scale=30
    ).get('slope')
    
    return mean_elevation, mean_slope

# Iterate over each reach in the reach_length_list and append elevation and slope
for reach in reach_length_list:
    reach_id = reach['REACH_ID']
    
    # Assuming you have a way to get the reach feature from the reach_id
    reach_feature = ee.Feature(aracataca_rivers.filter(ee.Filter.eq('REACH_ID', reach_id)).first())
    
    # Calculate mean elevation and slope for the reach
    elevation_value, slope_value = calculate_mean_elevation_slope(reach_feature)
    
    # Append the results to the reach
    reach['ELEVATION'] = int(elevation_value.getInfo()) if elevation_value else None
    reach['SLOPE'] = int(slope_value.getInfo()) if slope_value else None

# Print the results
for reach in reach_length_list:
    print(f"REACH_ID: {reach['REACH_ID']}, LENGTH_KM: {reach['LENGTH_KM']}, STRAIGHT_LINE_DISTANCE_KM: {reach['STRAIGHT_LINE_DISTANCE_KM']}, SINUOSITY: {reach['SINUOSITY']}, ELEVATION: {reach['ELEVATION']}, SLOPE: {reach['SLOPE']}")
    

REACH_ID: 60003284, LENGTH_KM: 3.509, STRAIGHT_LINE_DISTANCE_KM: 2.592474182207558, SINUOSITY: 1.353533247923031, ELEVATION: 1125, SLOPE: 19
REACH_ID: 60003308, LENGTH_KM: 0.65, STRAIGHT_LINE_DISTANCE_KM: 0.6486495434076742, SINUOSITY: 1.0020819510412837, ELEVATION: 1012, SLOPE: 19
REACH_ID: 60003348, LENGTH_KM: 2.665, STRAIGHT_LINE_DISTANCE_KM: 2.4586942533891714, SINUOSITY: 1.0839086626271028, ELEVATION: 938, SLOPE: 17
REACH_ID: 60003428, LENGTH_KM: 1.366, STRAIGHT_LINE_DISTANCE_KM: 1.3680906125703352, SINUOSITY: 0.9984718756556575, ELEVATION: 753, SLOPE: 25
REACH_ID: 60003038, LENGTH_KM: 1.158, STRAIGHT_LINE_DISTANCE_KM: 1.152182712876476, SINUOSITY: 1.0050489276210375, ELEVATION: 1732, SLOPE: 25
REACH_ID: 60003180, LENGTH_KM: 3.152, STRAIGHT_LINE_DISTANCE_KM: 2.911936217672379, SINUOSITY: 1.0824412914234478, ELEVATION: 1428, SLOPE: 26
REACH_ID: 60003037, LENGTH_KM: 0.683, STRAIGHT_LINE_DISTANCE_KM: 0.683399155571331, SINUOSITY: 0.9994159261566584, ELEVATION: 1735, SLOPE: 22
REACH_I

In [155]:
# Revised classification criteria; Maybe this needs to be recuded to less classes? Just four for now.
classifications = [
    {
        'name': 'Lowland Alluvial',
        'elevation': lambda ele: ele < 200,
        'slope': lambda slo: slo < 2,
        'sinuosity': lambda sin: sin > 1.4
    },
    {
        'name': 'Open-valley Lowland',
        'elevation': lambda ele: ele < 200,
        'slope': lambda slo:  2 <= slo <= 4,
        'sinuosity': lambda sin: sin > 1.2
    },
    {
        'name': 'Open-Valley Upland',
        'elevation': lambda ele: 200 <= ele <= 800,
        'slope': lambda slo: 2 <= slo <= 4,
        'sinuosity': lambda sin:  1.0 <= slo <= 1.2
    },
    {
        'name': 'Upland Constrained',
        'elevation': lambda ele: ele > 800,
        'slope': lambda slo: slo > 4,
        'sinuosity': lambda sin: 1.0 <= slo <= 1.1
    }
]

# Function to classify a reach
def classify_reach(reach):
    for classification in classifications:
        if (classification,'elevation' and
            classification,'slope' and
            classification,'sinuosity'):
            return classification['name']
    return 'Unknown'

# Classify each reach in the reach_length_list
for reach in reach_length_list:
    reach['CLASSIFICATION'] = classify_reach(reach)

# Print the results with classifications
for reach in reach_length_list:
    print(f"REACH_ID: {reach['REACH_ID']}, LENGTH_KM: {reach['LENGTH_KM']}, STRAIGHT_LINE_DISTANCE_KM: {reach['STRAIGHT_LINE_DISTANCE_KM']}, SINUOSITY: {reach['SINUOSITY']}, ELEVATION: {reach['ELEVATION']}, SLOPE: {reach['SLOPE']}, CLASSIFICATION: {reach['CLASSIFICATION']}")

REACH_ID: 60003284, LENGTH_KM: 3.509, STRAIGHT_LINE_DISTANCE_KM: 2.592474182207558, SINUOSITY: 1.353533247923031, ELEVATION: 1125, SLOPE: 19, CLASSIFICATION: Lowland Alluvial
REACH_ID: 60003308, LENGTH_KM: 0.65, STRAIGHT_LINE_DISTANCE_KM: 0.6486495434076742, SINUOSITY: 1.0020819510412837, ELEVATION: 1012, SLOPE: 19, CLASSIFICATION: Lowland Alluvial
REACH_ID: 60003348, LENGTH_KM: 2.665, STRAIGHT_LINE_DISTANCE_KM: 2.4586942533891714, SINUOSITY: 1.0839086626271028, ELEVATION: 938, SLOPE: 17, CLASSIFICATION: Lowland Alluvial
REACH_ID: 60003428, LENGTH_KM: 1.366, STRAIGHT_LINE_DISTANCE_KM: 1.3680906125703352, SINUOSITY: 0.9984718756556575, ELEVATION: 753, SLOPE: 25, CLASSIFICATION: Lowland Alluvial
REACH_ID: 60003038, LENGTH_KM: 1.158, STRAIGHT_LINE_DISTANCE_KM: 1.152182712876476, SINUOSITY: 1.0050489276210375, ELEVATION: 1732, SLOPE: 25, CLASSIFICATION: Lowland Alluvial
REACH_ID: 60003180, LENGTH_KM: 3.152, STRAIGHT_LINE_DISTANCE_KM: 2.911936217672379, SINUOSITY: 1.0824412914234478, ELEVAT

In [156]:
import folium

# Define color codes for each classification
classification_colors = {
    'Lowland Alluvial': 'blue',
    'Open-valley Lowland': 'green',
    'Open-Valley Upland': 'red',
    'Upland High-Energy': 'purple',
}

# Create a map centered on Aracataca
m = folium.Map(location=[10.5910, -74.1897], zoom_start=12)

# Add a legend to the map
legend_html = '''
<div style="position: fixed; 
            bottom: 50px; left: 50px; width: 200px; height: 200px; 
            border:2px solid grey; z-index:9999; font-size:14px;
            background-color:white; opacity: 0.8;">
    <h4>River Classifications</h4>
    <i style="background: blue"></i> Lowland Alluvial<br>
    <i style="background: green"></i> Open-valley Lowland<br>
    <i style="background: red"></i> Open-Valley Upland<br>
    <i style="background: purple"></i> Upland High-Energy<br>
</div>
'''

m.get_root().html.add_child(folium.Element(legend_html))

# Save the map to an HTML file
m.save('rivers_classification_map.html')

# Print a message indicating that the map has been created
print("The map with a legend for classifications has been created and saved as 'rivers_classification_map.html'.")

# Display the map
m

The map with a legend for classifications has been created and saved as 'rivers_classification_map.html'.
